In [1]:
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd


In [3]:
model = load_model('model.h5')

with open('label_encoder.pkl','rb') as file:
    label_encoder = pickle.load(file)

with open('onehot_encoder.pkl','rb')as file:
    onehot_encoder =pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [14]:
geo_encoded = onehot_encoder.transform([[input_data['Geography']]]).toarray()

geography_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))
geography_encoded_df

c:\Users\anjiu\Documents\AI_Learnings\Neural_Networks\Churn_classification_ANN\venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [17]:
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [19]:
input_df = pd.concat([input_df.drop("Geography", axis=1), geography_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
## scaling the input data

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step


array([[0.03594816]], dtype=float32)

In [22]:
predicition_probability = prediction[0][0]
predicition_probability


np.float32(0.03594816)

In [23]:
if predicition_probability > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
